how to solve this thing?

I need to do the process of a nural net. It's not that complicated but requiers a bit of finageling.

We start with splitting the data into train and validate sets.

Then we need to create the nural net that will run the calculations.

We will copy the pytorch interface

you have a Modle class, this encapuslates everything

